In [1]:
import os
import pandas as pd
import numpy as np
import sqlalchemy as sqla
from sqlalchemy.ext.automap import automap_base
import random as rnd

In [2]:
path = "DataSets/belly_button_biodiversity.sqlite"
engine = sqla.create_engine("sqlite:///"+path, echo=False)
Base = sqla.ext.automap.automap_base()
Base.prepare(engine, reflect=True)

# Binding the SQLITE engine to the current Session (locking file)
session = sqla.orm.Session(bind=engine)

In [3]:
# Verify tables and structure from SQLite database
inspector = sqla.inspect(engine)
tbl_names = inspector.get_table_names()
i = 0 
for table in tbl_names:
    print(table)
    i = i+1
print("___________________________\nTotal of %s tables" %i)

otu
samples
samples_metadata
___________________________
Total of 3 tables


In [4]:
OTU = Base.classes.otu
SAMPLES = Base.classes.samples
META = Base.classes.samples_metadata

In [5]:
OTU_COLS = []
columns = inspector.get_columns("otu")
for col in columns:
    OTU_COLS.append(col["name"])

In [6]:
SAMPLE_COLS = []
columns = inspector.get_columns("samples")
for col in columns:
    SAMPLE_COLS.append(col["name"])

In [7]:
META_COLS = []
columns = inspector.get_columns("samples_metadata")
for col in columns:
    META_COLS.append(col["name"])

In [48]:
OTU_df = pd.read_sql("SELECT * FROM otu", con=engine, columns=[OTU_COLS], index_col="otu_id")
SAMPLES_df = pd.read_sql("SELECT * FROM samples", con=engine, columns=[SAMPLE_COLS], index_col="otu_id")
META_ALL_df = pd.read_sql("SELECT * FROM samples_metadata", con=engine, columns=[META_COLS])
META_df = pd.read_sql("SELECT AGE, UPPER(BBTYPE) as BBTYPE, UPPER(ETHNICITY) as ETHNICITY, UPPER(GENDER) as GENDER, UPPER(LOCATION) as LOCATION, SAMPLEID FROM samples_metadata", con=engine, columns=[META_COLS])
META_WFREQ = pd.read_sql("SELECT WFREQ, SAMPLEID FROM samples_metadata", con=engine, columns=[META_COLS])

In [49]:
sample_names = SAMPLES_df.columns
sample_list_names = []
for col in sample_names:
    sample_list_names.append(col)

# @app.route("/names")
names = sample_list_names[0:]

In [50]:
otu_desc = []
for item in OTU_df["lowest_taxonomic_unit_found"]:
    otu_desc.append(item)

# @app.route("/otu")
otu = otu_desc

In [51]:
meta_ = META_df
meta_["index"]= "BB_"+meta_["SAMPLEID"].astype(str)
meta_.set_index("index", inplace=True)

# @app.route("/metadata/<sample>")
metadata = meta_.to_dict("index")

In [52]:
wfreq_ = META_WFREQ
wfreq_["index"] = "BB_"+wfreq_["SAMPLEID"].astype(str)
wfreq_.drop(["SAMPLEID"], axis=1, inplace=True)
wfreq_.set_index("index", inplace=True)

# @app.route("/wfreq/<sample>")
wfreq = wfreq_.to_dict("index")   #  wfreq["BB_940"]["WFREQ"]

In [53]:
otu_id_sort = OTU_df.sort_index(ascending=False)

In [54]:
sample_values = SAMPLES_df

In [58]:
metadata["BB_961"]

{'AGE': 35,
 'BBTYPE': 'I',
 'ETHNICITY': 'CAUCASIAN',
 'GENDER': 'F',
 'LOCATION': 'EDMONTON,AB',
 'SAMPLEID': 961}

In [37]:
META_ALL_df

SAMPLEID                      EVENT              ETHNICITY GENDER  AGE  \
0         940  BellyButtonsScienceOnline              Caucasian      F   24   
1         941                          0  Caucasian/Midleastern      F   34   
2         943  BellyButtonsScienceOnline              Caucasian      F   49   
3         944  BellyButtonsScienceOnline               European      M   44   
4         945  BellyButtonsScienceOnline              Caucasian      F   48   
5         946  BellyButtonsScienceOnline              Caucasian      F   42   
6         947  BellyButtonsScienceOnline              Caucasian      M   49   
7         948  BellyButtonsScienceOnline              Caucasian      M   20   
8         949  BellyButtonsScienceOnline              Caucasian      F   51   
9         950  BellyButtonsScienceOnline              Caucasian      M   51   
10        952  BellyButtonsScienceOnline              Caucasian      F   51   
11        953  BellyButtonsScienceOnline              Caucasian      F   38   
12        954  BellyButtonsScienceOnline              Caucasian      M   26   
13        955  BellyButtonsScienceOnline              Caucasian      F   27   
14        956  BellyButtonsScienceOnline              Caucasian      M   16   
15        958  BellyButtonsScienceOnline              Caucasian      M   43   
16        959  BellyButtonsScienceOnline              Caucasian      M   40   
17        960  BellyButtonsScienceOnline              Caucasian      M   48   
18        961  BellyButtonsScienceOnline              Caucasian      F   35   
19        962  BellyButtonsScienceOnline              Caucasian      F   56   
20        963  BellyButtonsScienceOnline              Caucasian      F   33   
21        964  BellyButtonsScienceOnline              Caucasian      F   43   
22        966  BellyButtonsScienceOnline              Caucasian      M   56   
23        967  BellyButtonsScienceOnline       Caucasian/Jewish      F   43   
24        968  BellyButtonsScienceOnline              Caucasian      F   38   
25        969  BellyButtonsScienceOnline        Caucasian/Asian      F   31   
26        970  BellyButtonsScienceOnline              Caucasian      F   42   
27        971  BellyButtonsScienceOnline           Asian(South)      M   36   
28        972  BellyButtonsScienceOnline              Caucasian      F   31   
29        973  BellyButtonsScienceOnline              Caucasian      F    4   
..        ...                        ...                    ...    ...  ...   
123      1540      BellyButtonsWildLives              Caucasian      M   30   
124      1541      BellyButtonsWildLives              Caucasian      F   26   
125      1542      BellyButtonsWildLives              Caucasian      M   36   
126      1543      BellyButtonsWildLives              Caucasian      F   34   
127      1544      BellyButtonsWildLives              Caucasian      F   33   
128      1545      BellyButtonsWildLives              Caucasian      F   39   
129      1546      BellyButtonsWildLives              Caucasian      M   34   
130      1547      BellyButtonsWildLives                      0      M   38   
131      1548      BellyButtonsWildLives              Caucasian      M    6   
132      1549      BellyButtonsWildLives              Caucasian      F   37   
133      1550      BellyButtonsWildLives              Caucasian      F   29   
134      1551      BellyButtonsWildLives              Caucasian      M   26   
135      1552      BellyButtonsWildLives              Caucasian      M   25   
136      1553      BellyButtonsWildLives              Caucasian      F   50   
137      1554      BellyButtonsWildLives              Caucasian      F   18   
138      1555      BellyButtonsWildLives              Caucasian      M   26   
139      1556      BellyButtonsWildLives               Hispanic      M   34   
140      1557      BellyButtonsWildLives              Caucasian      F   26   
141      1558      BellyButtonsWildLives              C

In [ ]:
wfreq